In [1]:
from constants import DEV_BUCKET, STAGE_BUCKET, PROD_BUCKET, FOLDER_NAME
import tempfile
from dvc.api import DVCFileSystem
import s3fs
import yaml
import datetime

In [2]:
repo = 'git@github.com:ContextLogic/multitask-llm-rnd.git'
path = '/modelling/notebooks/convert_pl_to_hf_ckpt/text_emb_v1/onnx_ckpt'

In [3]:
MODEL_NAME = "text-emb/v1"

In [4]:
s3_file = s3fs.S3FileSystem()

In [5]:
with tempfile.TemporaryDirectory() as tmpdirname:
    print(tmpdirname)
    fs = DVCFileSystem(repo, subrepos=True)
    fs.get(path, tmpdirname, recursive=True)
    s3_file = s3fs.S3FileSystem()
    local_path = tmpdirname
    
    def rm_and_save(s3_path):
        if len(s3_file.ls(s3_path)) > 0:
            try:
                print(f'rm -rf {s3_path}')
                s3_file.rm(s3_path, recursive=True)
            except Exception as e:
                print(f'rm failed due to {e}')
        print(f'save to {s3_path}')
        s3_file.put(local_path, s3_path, recursive=True) 

    rm_and_save(f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    rm_and_save(f"{STAGE_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    rm_and_save(f"{PROD_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    

/tmp/tmpi3g6sz94
save to structured-data-dev/coeus-gpu-multitask-ml/text-emb/v1
save to structured-data-stage/coeus-gpu-multitask-ml/text-emb/v1
save to structured-data-prod/coeus-gpu-multitask-ml/text-emb/v1


In [6]:
yaml_dict = yaml.safe_load(open('models.yaml', 'r'))

In [7]:
if yaml_dict is None:
    yaml_dict = {}

In [8]:
assert MODEL_NAME.count('/') == 1

In [9]:
model_class, model_version = MODEL_NAME.split('/')

In [10]:
if model_class not in yaml_dict:
    yaml_dict[model_class] ={}
if model_version in yaml_dict[model_class]:
    print(f'overwrite {model_class}/{model_version} !!!!!')
yaml_dict[model_class][model_version] = {
    "dvc": {
        "repo": "git@github.com:ContextLogic/multitask-llm-rnd.git", 
        "path": "/modelling/notebooks/convert_pl_to_hf_ckpt/text_emb_v1/onnx_ckpt", 
        "rev": "45340f4f9d957169d95ad48f884cf531d0c36fe8"
    },
    "s3": {
        "dev": f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}",
        "stage": f"{STAGE_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}",
        "prod": f"{PROD_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}"
    },
    "metadata": {
        "library_type": "onnx",
        "model_type": "text-embedding",
        "yaml_update_time": str(datetime.datetime.now())
    }
}

In [11]:
with open('models.yaml', 'w') as f:
    f.write(yaml.dump(yaml_dict))

In [12]:
import s3fs
fs = s3fs.S3FileSystem()
fs.get(f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}/", "tmp", recursive=True)